In [60]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np
from torch.utils.data import TensorDataset


In [34]:
df = pd.read_parquet("small_tokenized_data.parquet")

In [35]:
print(df.head())
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

   stars                                               text  \
0      2  If you decide to eat here, just be aware it is...   
1      4  I've taken a lot of spin classes over the year...   
2      2  Family diner. Had the buffet. Eclectic assortm...   
3      4  Wow!  Yummy, different,  delicious.   Our favo...   
4      3  Cute interior and owner (?) gave us tour of up...   

                                           input_ids  \
0  [101, 2065, 2017, 5630, 2000, 4521, 2182, 1010...   
1  [101, 1045, 1005, 2310, 2579, 1037, 2843, 1997...   
2  [101, 2155, 15736, 1012, 2018, 1996, 28305, 10...   
3  [101, 10166, 999, 9805, 18879, 1010, 2367, 101...   
4  [101, 10140, 4592, 1998, 3954, 1006, 1029, 100...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4  [

In [68]:
#get the attention mask
attn_mask_list = df["attention_mask"].tolist()
attn_mask=[]
for each in attn_mask_list:
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    attn_mask.append(each)

In [69]:
#get the star count AKA the y from half_data
stars = df["stars"].tolist()
starIDS=[]
for each in stars:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    starIDS.append(each)

In [70]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [73]:
#load data in batches for bert
bertData = TensorDataset(torch.tensor(newIDS), torch.tensor(attn_mask), torch.tensor(starIDS))
# set up a data loader to get batches from our dataset
bertLoader = torch.utils.data.DataLoader(bertData, batch_size = 16, shuffle = True)

In [ ]:
i = 0
for batch in bertLoader:
    y, pooled = bert(batch[0])
    print(pooled.shape)
    print(pooled)

    torch.save(pooled, ("vector"+"%d"+".txt")%i)
        
    torch.save(batch[1], ("label"+"%d"+".txt")%i)

    print(i)
    i = i+1

torch.Size([16, 768])
tensor([[-0.7095, -0.7251, -0.9989,  ..., -0.9867, -0.7343,  0.4843],
        [-0.4864, -0.6547, -0.9994,  ..., -0.9906, -0.5134,  0.0553],
        [-0.5880, -0.6714, -0.9982,  ..., -0.9911, -0.6021,  0.2036],
        ...,
        [-0.3518, -0.4987, -0.9953,  ..., -0.9548, -0.3662, -0.0789],
        [-0.6950, -0.7400, -0.9994,  ..., -0.9914, -0.6979,  0.4265],
        [-0.6581, -0.7117, -0.9995,  ..., -0.9957, -0.7217,  0.4861]],
       grad_fn=<TanhBackward0>)
0
torch.Size([16, 768])
tensor([[-0.4350, -0.6582, -0.9986,  ..., -0.9881, -0.4199, -0.1506],
        [-0.4150, -0.4869, -0.9962,  ..., -0.9677, -0.4544, -0.1006],
        [-0.4036, -0.7195, -0.9978,  ..., -0.9821, -0.6394,  0.2457],
        ...,
        [-0.4309, -0.5282, -0.9944,  ..., -0.9701, -0.4067,  0.3970],
        [-0.6601, -0.6068, -0.9959,  ..., -0.9822, -0.5707,  0.4432],
        [-0.5120, -0.6687, -0.9953,  ..., -0.9637, -0.5847,  0.1854]],
       grad_fn=<TanhBackward0>)
1
torch.Size([16, 768]

In [ ]:
#...final_pool = np.concatinate(list_pool/files)